# League of Legends Analysis
# Vy Nguyen and Daniel Strub
# CPSC 322

In [17]:
import importlib
import sys
import os
import copy
from tabulate import tabulate

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [15]:
game_fname = os.path.join("input_data", "games_small.csv")
game_table = MyPyTable()
game_table.load_from_file(game_fname)

game_ID = game_table.get_column("gameId")
winner = game_table.get_column("winner")
first_Blood = game_table.get_column("firstBlood")
first_Tower = game_table.get_column("firstTower")
first_Inhibitor = game_table.get_column("firstInhibitor")
first_Baron = game_table.get_column("firstBaron")
first_Dragon = game_table.get_column("firstDragon")
first_RiftHerald = game_table.get_column("firstRiftHerald")

game_X = [[first_Blood[x], first_Tower[x], first_Inhibitor[x], first_Baron[x], first_Dragon[x], first_RiftHerald[x]] for x in range(len(game_ID))]
game_Y = [y for y in game_ID]


In [19]:
print(str(myutils.get_win_count(game_table, "winner", "firstBlood") * 100) + "% of games with first blood are won")

print(str(myutils.get_win_count(game_table, "winner", "firstTower") * 100) + "% of games with first tower are won")

print(str(myutils.get_win_count(game_table, "winner", "firstInhibitor") * 100) + "% of games with first inhibitor are won")

print(str(myutils.get_win_count(game_table, "winner", "firstBaron") * 100) + "% of games with first Baron are won")

print(str(myutils.get_win_count(game_table, "winner", "firstDragon") * 100) + "% of games with first dragon are won")

print(str(myutils.get_win_count(game_table, "winner", "firstRiftHerald") * 100) + "% of games with first Rift Herald are won")

58.55723714914327% of gmaes with first blood are won
68.9245949729982% of gmaes with first tower are won
79.83198879925328% of gmaes with first inhibitor are won
49.09660644042936% of gmaes with first Baron are won
65.60437362490833% of gmaes with first dragon are won
34.00226681778785% of gmaes with first Rift Herald are won


In [23]:
# KNNeightbors
test_size = 10
X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(game_X), copy.deepcopy(game_Y), test_size=test_size, shuffle=True)

nb = MyKNeighborsClassifier()
nb.fit(X_train, y_train)
predictions = []
for i, x in enumerate(X_test):
    prediction = nb.predict([x])
    predictions.append(prediction[0])

acc = round(sum([int(x==y) for x,y in zip(predictions, y_test)])/len(predictions), 2)
print("KNN:" )
print("Accuracy = " + str(acc))
print("Error rate = " + str(1-acc))

KNN:
Accuracy = 0.0
Error rate = 1.0


In [24]:
nb_predicted = []
nb_actual = []
# Stratified kfold cross validation
X_train_folds, X_test_folds = myevaluation.stratified_kfold_cross_validation(game_X, game_Y, n_splits=10)
for i in range(0, 1):
    X_train, X_test, y_train, y_test = myutils.get_trains_and_tests(game_X, game_Y, X_train_folds[i], X_test_folds[i])
    nb = MyKNeighborsClassifier()
    nb.fit(X_train, y_train)
    predictions = []
    for i, x in enumerate(X_test):
        prediction = nb.predict([x])
        predictions.append(prediction[0])
    predicted = [ x for x in predictions]
    actual = [x for x in y_test]
    
    nb_predicted +=  predicted
    nb_actual += actual
# Result
acc = round(sum([int(x==y) for x,y in zip(predicted, actual)])/len(predicted), 2)
print("Naive Bayes: ")
print("Accuracy = " + str(acc))
print("Error rate = " + str(1-acc))

Naive Bayes: 
Accuracy = 0.0
Error rate = 1.0


In [14]:
# Decision Tree for now
test_size = 10
predicted_tree = []
actual_tree = []

# Get training data with stratified kfold cross validation
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(game_X, game_Y, test_size)
for i in range(test_size):
    # Sort training data
    xtrain = myutils.distribute_data_by_index(X_train, train_folds[i])
    ytrain = myutils.distribute_data_by_index(Y_train, train_folds[i])
    xtest = myutils.distribute_data_by_index(X_train, test_folds[i])
    ytest = myutils.distribute_data_by_index(Y_train, test_folds[i])

    # Compute prediction and convert
    tree = MyDecisionTreeClassifier()
    tree.fit(xtrain, ytrain)
    predicted = tree.predict(xtest)
    predicted_tree += predicted
    actual_tree += ytest
    
# Result
acc = round(sum([int(x==y) for x,y in zip(predicted_tree, actual_tree)])/len(predicted_tree), 2)
error_rate = 1- accuracy

print("Decision Tree: ")
print("Accuracy = " + str(acc) )
print("Error Rate = " + str(1-acc))


IndexError: list index out of range